In [7]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd
import numpy as np
import random

In [14]:
data = pd.DataFrame()

animal = ['Dog']*200 + ['Cat']*100 + ['Frog']*100
random.shuffle(animal)
data['Animal'] = animal

age = np.random.randint(12, 50, size=400)
data['Age'] = age

healthy = np.random.randint(0,1,size=400)
data['Healthy'] = healthy

data.head()

,Animal,Age,Healthy
0,Dog,38,0
1,Frog,38,0
2,Cat,38,0
3,Dog,26,0
4,Frog,32,0
